In [1]:
import numpy as np
import pandas as pd
from typing import Tuple
from scipy.optimize import minimize
import yfinance as yf

# ============================================================
# 1) PREISDATEN LADEN
# ============================================================

tickers = ["AAPL", "MSFT", "GOOG"]

data = yf.download(tickers, period="2y", auto_adjust=False)

# Spalte auswählen: Adj Close oder Close
if "Adj Close" in data.columns.get_level_values(0):
    adj_close = data["Adj Close"]
else:
    adj_close = data["Close"]

print("Preisdaten geladen:\n", adj_close.tail())

# ============================================================
# 2) RENDITEN BERECHNEN
# ============================================================

returns_df = adj_close.pct_change().dropna()
print("\nRenditen geladen:")
print(returns_df.head())
print("Shape:", returns_df.shape)

returns_array = returns_df.values  # NumPy-Array

# ============================================================
# 3) MEAN-VARIANCE KRITERIUM
# ============================================================

def mv_criterion(
    weights: np.ndarray,
    returns: np.ndarray,
    risk_aversion: float = 3.0,
    initial_wealth: float = 1.0,
    excess_return_target: float = 0.25 / 100,
) -> float:

    w = np.asarray(weights, dtype=float).reshape(-1)
    r = np.asarray(returns, dtype=float)

    lam = float(risk_aversion)
    W0 = float(initial_wealth)
    W_bar = W0 * (1 + excess_return_target)

    port_ret = r @ w
    mu = np.mean(port_ret)
    sigma = np.std(port_ret, ddof=1)

    term1 = W_bar ** (1 - lam) / (1 + lam)
    term2 = W_bar ** (-lam) * W0 * mu
    term3 = lam / 2 * W_bar ** (-1 - lam) * W0**2 * sigma**2

    utility = term1 + term2 - term3

    return -utility  # Minimierer soll maximieren


def mv_criterion_with_stats(weights, returns, **kwargs) -> Tuple[float, float, float]:
    w = np.asarray(weights, dtype=float)
    r = np.asarray(returns, dtype=float)

    port_ret = r @ w
    mu = np.mean(port_ret)
    sigma = np.std(port_ret, ddof=1)

    value = mv_criterion(w, r, **kwargs)
    return value, mu, sigma

# ============================================================
# 4) OPTIMIERUNG
# ============================================================

n_assets = returns_array.shape[1]
w0 = np.ones(n_assets) / n_assets

constraints = {"type": "eq", "fun": lambda w: np.sum(w) - 1}
bounds = [(0, 1)] * n_assets

result = minimize(
    mv_criterion,
    w0,
    args=(returns_array,),
    method="SLSQP",
    constraints=constraints,
    bounds=bounds,
)

if not result.success:
    print("Optimierung NICHT erfolgreich:", result.message)
else:
    opt_w = result.x
    value, mu_opt, sigma_opt = mv_criterion_with_stats(opt_w, returns_array)

    print("\nOptimierung erfolgreich!")
    print("Optimale Gewichte:", opt_w)
    print("Erwartete Rendite:", mu_opt)
    print("Volatilität:", sigma_opt)


[*********************100%***********************]  3 of 3 completed

Preisdaten geladen:
 Ticker            AAPL        GOOG        MSFT
Date                                          
2025-12-01  283.100006  315.119995  486.739990
2025-12-02  286.190002  316.019989  490.000000
2025-12-03  284.149994  320.619995  477.730011
2025-12-04  280.700012  318.390015  480.839996
2025-12-05  278.779999  322.089996  483.160004

Renditen geladen:
Ticker          AAPL      GOOG      MSFT
Date                                    
2023-12-07  0.010139  0.053413  0.005830
2023-12-08  0.007412 -0.013073  0.008842
2023-12-11 -0.012928 -0.014198 -0.007830
2023-12-12  0.007920 -0.007869  0.008295
2023-12-13  0.016691  0.002469 -0.000027
Shape: (501, 3)

Optimierung erfolgreich!
Optimale Gewichte: [0.33333333 0.33333333 0.33333333]
Erwartete Rendite: 0.001186985493803312
Volatilität: 0.013564574107561643


In [2]:
# Annahme: returns_df ist ein DataFrame mit Renditen (z.B. adj_close.pct_change().dropna())
data = returns_df.values   # (n_periods, n_assets)

# Train-/Test-Split
split = int(0.7 * len(data))
train_set = data[:split, :]   # In-Sample
test_set = data[split:, :]    # Out-of-Sample (für spätere Auswertung)

# Anzahl der Assets
n = train_set.shape[1]

# Startgewichte (gleichgewichtet, Summe = 1)
x0 = np.ones(n) / n

# Nebenbedingung: Summe der Gewichte = 1
cons = {'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0}

# Bounds: keine Short-Sales, Gewichte zwischen 0 und 1
Bounds = [(0.0, 1.0)] * n

# Optimierung
res_MV = minimize(
    mv_criterion,       # unsere Funktion von oben
    x0,
    args=(train_set,),  # wichtig: Komma, damit es ein Tupel ist
    method="SLSQP",
    bounds=Bounds,
    constraints=[cons],
    options={'disp': True}
)

X_MV = res_MV.x  # optimale Gewichte (In-Sample)
print("Optimale Gewichte (Train):", X_MV)


Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.24904156231265082
            Iterations: 1
            Function evaluations: 4
            Gradient evaluations: 1
Optimale Gewichte (Train): [0.33333333 0.33333333 0.33333333]
